In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# gerais

import pandas as pd
from tqdm import tqdm


# Gensim
from nltk import tokenize
import gensim.corpora as corpora
from gensim.models import CoherenceModel

#plot
import matplotlib
matplotlib.use("TkAgg")  # Define o backend ANTES de importar pyplot

import matplotlib.pyplot as plt  # Só importar depois de definir o backend

import pyLDAvis

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from unidecode import unidecode


from string import punctuation

from gensim.models import LdaMulticore, CoherenceModel


In [3]:
df = pd.read_csv('../datasets/fakeWhatsApp.BR_2022.csv')

In [4]:
len(df)

598971

In [23]:
minimo = df['date_message'].min()
maximo = df['date_message'].max()

print(maximo)
print(minimo)

2022-11-14 09:13:45
2022-09-27 00:00:08


In [24]:
df.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,5,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,5,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
2,2022-10-11 00:39:31,c882172d447798d74915973ac83eba68,b84dfe2d1599b82768dcdecce7e6bb23,d2e0ec59ffd9f84764f5b147725d7196.oga,audio/ogg; codecs=opus,NaN,True,False,False,NaN,5,2022-10-11 00:39:33.445125,NaN,NaN,737948BE86D450A426470794F91BC80D,558594228826.0:12@s.whatsapp.net,Audio,whatsapp,NaN,d2e0ec59ffd9f84764f5b147725d7196
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,5,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
4,2022-10-10 23:40:12,77c1a8a31fee269db258a028a61f0b88,a5831b109d4d95fae8ee0ce464e48a6a,bb80cd530332bb6b95c34e719992d249.f4v,video/mp4,NaN,True,False,False,NaN,5,2022-10-10 23:40:14.397495,NaN,NaN,092203A082AC3DFB2A4933F60453AEB8,558594228826.0:12@s.whatsapp.net,Video,whatsapp,NaN,bb80cd530332bb6b95c34e719992d249


In [25]:
df.columns

Index(['date_message', 'id_member_anonymous', 'id_group_anonymous', 'media',
       'media_type', 'media_url', 'has_media', 'has_media_url', 'trava_zap',
       'text_content_anonymous', 'dataset_info_id', 'date_system',
       'score_sentiment', 'score_misinformation', 'id_message', 'id_persona',
       'message_type', 'messenger', 'media_name', 'media_md5'],
      dtype='object')

In [5]:
ids_para_remover = ['6ef561ec0f448afcd7b3751124bb0712','d3e678a0ba0e1485548260a7c4599152',
                    'ac5703154484de05336af617455ca55e','a39edbd64d378226ffa60433649a0acf',
                    'c66d0d4ae5a4b281bff67e1fa4fbd6ba','819bbc872ed6d81f44d746b710eecf06',
                    'ec94da4d54f9a5693e88fa582926be53','6a38c72316d87c028dfd66c10442476b',
                    '94099e1e46f129856541e2b3640896d1','8f367d1693fff47218603fa47ded525c',
                    'c09caffee0d1bd30926dea9df25dc88f']

In [6]:
# Filtrar o DataFrame para remover as linhas com esses IDs
df_filtrado = df[~df['id_member_anonymous'].isin(ids_para_remover)]
df_filtrado

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,5,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,5,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
2,2022-10-11 00:39:31,c882172d447798d74915973ac83eba68,b84dfe2d1599b82768dcdecce7e6bb23,d2e0ec59ffd9f84764f5b147725d7196.oga,audio/ogg; codecs=opus,NaN,True,False,False,NaN,5,2022-10-11 00:39:33.445125,NaN,NaN,737948BE86D450A426470794F91BC80D,558594228826.0:12@s.whatsapp.net,Audio,whatsapp,NaN,d2e0ec59ffd9f84764f5b147725d7196
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,5,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
4,2022-10-10 23:40:12,77c1a8a31fee269db258a028a61f0b88,a5831b109d4d95fae8ee0ce464e48a6a,bb80cd530332bb6b95c34e719992d249.f4v,video/mp4,NaN,True,False,False,NaN,5,2022-10-10 23:40:14.397495,NaN,NaN,092203A082AC3DFB2A4933F60453AEB8,558594228826.0:12@s.whatsapp.net,Video,whatsapp,NaN,bb80cd530332bb6b95c34e719992d249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598966,2022-11-13 00:46:56,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,7719ba57cbe74eb5c8059d6b1a14d1ca.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.262572,NaN,NaN,08E682716448AB06DEE126A30E7AAA77,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,7719ba57cbe74eb5c8059d6b1a14d1ca
598967,2022-11-13 13:37:03,2ad743e9ff831a75911a983b42205845,64c9a7c8ac338faa258e5aea9a1c2ae3,c6d59f0b86e63dbcce7fc682a1eaea50.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.269632,NaN,NaN,3EB0259ED9B8FACF18A9,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,c6d59f0b86e63dbcce7fc682a1eaea50
598968,2022-11-13 17:43:22,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,003ace942d9665adc9647df9d9526c0c.jpeg,image/jpeg,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.277283,NaN,NaN,5CD9F23CD07BEE38693E68E11F9C6A55,558594228826.0:17@s.whatsapp.net,Imagem,whatsapp,NaN,003ace942d9665adc9647df9d9526c0c
598969,2022-11-13 21:40:47,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,bab5653a94b097404739a061dbbc7407.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.284021,NaN,NaN,D3BDE4A5C68E7148C0925F6C582C99A9,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,bab5653a94b097404739a061dbbc7407


In [7]:
# Removendo linhas com valores NaN na coluna 'text_content_anonymous'
df_filtrado = df_filtrado.dropna(subset=['text_content_anonymous'])

# Removendo trava_zap
df_filtrado = df_filtrado[df_filtrado['trava_zap'] == False]



#df_filtrado = df_filtrado[0:100]
len(df_filtrado)

254480

In [8]:
token_espaco = tokenize.WhitespaceTokenizer()
token_pontuacao = tokenize.WordPunctTokenizer()

In [9]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):

    # Função para extrair e substituir o domínio da URL
    def substituir_dominios(texto):
        # Função para extrair e substituir o domínio da URL
        def extrair_dominio(url):
            # Remove o protocolo (http://, https://, etc.) e o "www." se presente
            dominio = re.sub(r'^https?://(?:www\.)?|www\.', '', url)
            # Remove o caminho e parâmetros da URL
            dominio = re.split(r'[/?#]', dominio)[0]
            # Retorna a parte principal do domínio (antes do primeiro ponto)
            return dominio.split('.')[0]

        # Substitui URLs por seus domínios principais
        return re.sub(r'https?://(?:www\.)?\S+|www\.\S+', lambda match: extrair_dominio(match.group(0)), texto)

    # Substituir domínios
    text = substituir_dominios(text)

    # Converte para minúsculas
    text = text.lower()

    # Remove acentos
    text = unidecode(text)

    #Remover Pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove URLs e menções
    #text = re.sub(r'http\S+|www\S+|https\S+|@\w+', '', text)

    # Substitui emojis repetidos por apenas um
    text = re.sub(r'([\U00010000-\U0010FFFF])\1+', r'\1', text)
    text = re.sub(r'([\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F780-\U0001F7FF]|[\U0001F800-\U0001F8FF]|[\U0001F900-\U0001F9FF]|[\U0001FA00-\U0001FA6F]|[\U0001FA70-\U0001FAFF])\1+', r'\1', text)


    # Remove espaços em branco extras (início ou final) e múltiplos espaços no meio do texto
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove pontuações e caracteres especiais
    #text = re.sub(r'[^\w\s]', '', text)

    # Ajusta risadas "kkk" ou mais para "kk"
    text = re.sub(r'k{2,}|K{2,}', 'kk', text)

    # Ajusta risadas "haha" ou mais para "haha"
    text = re.sub(r'(ha){2,}', 'haha', text, flags=re.IGNORECASE)

    # Ajusta risadas "kaka" ou mais para "kaka"
    text = re.sub(r'(ka){2,}', 'kaka', text, flags=re.IGNORECASE)

    # Remove as stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])


    return text


[nltk_data] Downloading package punkt to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:

df_geral = df_filtrado.copy()

# Aplicar o pré-processamento à coluna de texto
df_geral['text_processed'] = df_geral['text_content_anonymous'].apply(preprocess_text)

In [32]:
df_geral.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5,text_processed
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,...,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN,hoje fique sabendo canal lula flix vi pt entro...
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,...,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,pra nao aconteca nois fazer anossa parte
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,...,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,acesse link entrar grupo whatsapp chat
7,2022-10-10 16:49:48,325720ed3339a91b8076df12c1e95c45,0e345813dcb62b0fe4d8537f311af0f1,NaN,NaN,https://m.kwai.com/photo/150000006567403/52240...,False,True,False,Fortes palavras da Pastora Damares!\nhttps://k...,...,2022-10-10 16:49:50.051126,0.0000,NaN,F18BADED5AFA8FB0C33FE36625872DB8,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,fortes palavras pastora damares kwaivideo
8,2022-10-10 16:49:53,8ed44a70a011285622e2b8919c2c8c3e,3b3e64e81c3e3f3bfc5ba76ddb90fb2d,0d6f2896e5941ef1933e69bbd7a32f69.jpeg,image/jpeg,NaN,True,False,False,🤣🤣🤣🤣🤣🤣🤣🤣,...,2022-10-10 16:49:54.713831,0.0000,NaN,724C233591C3A62A5231B47FF22ADE4B,558594228826.0:12@s.whatsapp.net,Imagem,whatsapp,NaN,0d6f2896e5941ef1933e69bbd7a32f69,


### Calculando a diversidade dos tópicos
A diversidade de tópicos mede quantas palavras únicas existem entre os top-N termos de todos os tópicos. 

In [11]:

def topic_diversity(model, top_n=10):
    topic_words = []
    for k in range(model.num_topics):
        words = [word for word, _ in model.show_topic(k, topn=top_n)]
        topic_words.extend(words)
    unique_words = set(topic_words)
    return len(unique_words) / (top_n * model.num_topics)


### IRBO 

IRBO avalia quanto os tópicos são distintos entre si, usando o Rank-Biased Overlap (RBO) invertido. Quanto maior o IRBO, mais diferentes são os tópicos

In [12]:
import rbo

def compute_irbo(model, top_n=10):
    topics = []
    for k in range(model.num_topics):
        topic_terms = [word for word, _ in model.show_topic(k, topn=top_n)]
        topics.append(topic_terms)

    # Calcular média dos RBOs invertidos entre todos os pares
    n = len(topics)
    total_irbo = 0
    count = 0

    for i in range(n):
        for j in range(i + 1, n):
            rbo_score = rbo.RankingSimilarity(topics[i], topics[j]).rbo()
            total_irbo += (1 - rbo_score)
            count += 1

    return total_irbo / count if count > 0 else 0


In [13]:
def processar_texto(texto):
    texto = texto.lower()
    palavras_texto = token_espaco.tokenize(texto)
    palavras_texto = token_pontuacao.tokenize(' '.join(palavras_texto))
    return [palavra for palavra in palavras_texto]

texto_processado = df_geral['text_processed'].apply(processar_texto)

In [14]:
# Criar dicionário
id2word = corpora.Dictionary(texto_processado)
# Criar o Corpus: A Frequência das palavras nos documentos
corpus = [id2word.doc2bow(text) for text in texto_processado]

## Modelando os Tópicos sem filtro

In [15]:

# Função para determinar a melhor quantidade de tópicos para a modelagem
def compute_coherence_values(dictionary, corpus, texts, coherence, limit=30, start=2, step=5):
    coherence_values = []
    model_list = []
    topic_range = list(range(start, limit, step))

    for num_topics in tqdm(topic_range, desc=f"Calculando coerência ({coherence})"):
        model = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=5, workers=4)
        model_list.append(model)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    optimal_index = coherence_values.index(max(coherence_values))
    optimal_num_topics = topic_range[optimal_index]
    return optimal_num_topics, model_list, coherence_values

# Calcular os valores de coerência para diferentes métricas
def calculate_coherence_for_metrics(dictionary, corpus, texts, metrics, start=2, limit=30, step=2):
    coherence_results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_values(
            dictionary=dictionary,
            corpus=corpus,
            texts=texts,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        coherence_results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return coherence_results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

# Calcular os valores de coerência 
coherence_results = calculate_coherence_for_metrics(
    dictionary=id2word,
    corpus=corpus,
    texts=texto_processado,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)

# Plotar os resultados de coerência para cada métrica
for metric in metrics:
    x = list(range(2, 30, 2))
    coherence_values = coherence_results[metric]['coherence_values']
    plt.plot(x, coherence_values, marker='o', label=metric.upper())

plt.xlabel("Número de Tópicos")
plt.ylabel("Score de Coerência")
plt.legend(loc='best')
plt.title("Coerência por Métrica")
plt.grid(True)
plt.tight_layout()
plt.show()

# Mostrar os resultados de coerência para cada métrica
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {coherence_results[metric]['num_topics']}")


Calculando coerência (c_v):   0%|          | 0/14 [00:00<?, ?it/s]c:\Users\Melissa Felipe\.conda\envs\topics_env\lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  upcast = np.find_common_type(args, [])
c:\Users\Melissa Felipe\.conda\envs\topics_env\lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  upcast = np.find_common_type(args, [])
c:\Users\Melissa Felipe\.conda\envs\topics_env\lib\site-packages\scipy\sparse\_sputils.py:44: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy


Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.5746
Qtd. tópicos = 4 -> Coerência = 0.5014
Qtd. tópicos = 6 -> Coerência = 0.5269
Qtd. tópicos = 8 -> Coerência = 0.4739
Qtd. tópicos = 10 -> Coerência = 0.4687
Qtd. tópicos = 12 -> Coerência = 0.4558
Qtd. tópicos = 14 -> Coerência = 0.4912
Qtd. tópicos = 16 -> Coerência = 0.4835
Qtd. tópicos = 18 -> Coerência = 0.4581
Qtd. tópicos = 20 -> Coerência = 0.4725
Qtd. tópicos = 22 -> Coerência = 0.454
Qtd. tópicos = 24 -> Coerência = 0.4582
Qtd. tópicos = 26 -> Coerência = 0.463
Qtd. tópicos = 28 -> Coerência = 0.4861
Melhor número de tópicos segundo C_V: 2

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -5.6202
Qtd. tópicos = 4 -> Coerência = -5.1687
Qtd. tópicos = 6 -> Coerência = -4.8847
Qtd. tópicos = 8 -> Coerência = -4.4796
Qtd. tópicos = 10 -> Coerência = -4.6612
Qtd. tópicos = 12 -> Coerência = -4.5572
Qtd. tópicos = 14 -> Coerência = -4.4436
Qtd. tópicos = 16 -> Coerência = -3.6166
Qtd. tópi

In [16]:
# tamanho do corpus
len(corpus)


254480

In [17]:

pyLDAvis.enable_notebook()  


# Escolher a métrica para visualizar
chosen_metric = 'u_mass'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.214867  0.102822       1        1  13.164131
5     -0.197310  0.102721       2        1  11.909390
11    -0.144346 -0.047245       3        1  10.619098
13    -0.039595 -0.168274       4        1   7.786409
14    -0.130981  0.021129       5        1   7.509046
3     -0.146748  0.011525       6        1   7.041704
10     0.144542 -0.064150       7        1   6.825544
4      0.091393 -0.215163       8        1   5.805860
0      0.111798 -0.210707       9        1   4.748877
2     -0.065842  0.076545      10        1   4.664167
6     -0.039940 -0.011410      11        1   4.155566
12    -0.045024 -0.069958      12        1   4.039887
7      0.298002  0.180039      13        1   3.397288
15     0.255312  0.086942      14        1   3.187224
1      0.069026  0.014963      15        1   2.650619
9      0.054582  0.190222      16        1   2.495191, topic_info=                 Term          Freq         Total Category  logprob  loglift
3607               13  59788.000000  59788.000000  Default  30.0000  30.0000
4005               55  57036.000000  57036.000000  Default  29.0000  29.0000
8480            phone  56155.000000  56155.000000  Default  28.0000  28.0000
75428  99999999999999  26599.000000  26599.000000  Default  27.0000  27.0000
279         bolsonaro  60112.000000  60112.000000  Default  26.0000  26.0000
...               ...           ...           ...      ...      ...      ...
231               vai   1006.388373  25080.469148  Topic16  -5.0399   0.4751
9                lula    931.067301  56443.985804  Topic16  -5.1177  -0.4139
279         bolsonaro    900.122859  60112.658003  Topic16  -5.1515  -0.5106
51             brasil    382.485571  48667.231053  Topic16  -6.0074  -1.1553
464             ainda    362.856988   8611.047591  Topic16  -6.0601   0.5240

[1180 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
27243     13  0.992478  0000
46         1  0.000538     1
46         2  0.024206     1
46         3  0.022928     1
46         4  0.007127     1
...      ...       ...   ...
5113       3  0.016211  zona
5113       7  0.093216  zona
5113       8  0.099848  zona
5113       9  0.193801  zona
5113      12  0.450974  zona

[3929 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 6, 12, 14, 15, 4, 11, 5, 1, 3, 7, 13, 8, 16, 2, 10])

In [18]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 0.7625
IRBO médio: 0.9585

Perplexidade:  -9.691065365988242


In [19]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppSemFiltro_u_mass.csv', index=False, encoding='utf-8')


In [20]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_v'  # ou 'u_mass', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.232719  0.0       1        1  75.895582
1     -0.232719  0.0       2        1  24.104418, topic_info=                 Term          Freq         Total Category  logprob  loglift
3607               13  55119.000000  55119.000000  Default  30.0000  30.0000
4005               55  49445.000000  49445.000000  Default  29.0000  29.0000
8480            phone  47165.000000  47165.000000  Default  28.0000  28.0000
39              youtu  39163.000000  39163.000000  Default  27.0000  27.0000
75428  99999999999999  20715.000000  20715.000000  Default  26.0000  26.0000
...               ...           ...           ...      ...      ...      ...
690                sp   4196.446459   5121.565122   Topic2  -5.8801   1.2236
245              user   4104.052090   5802.436187   Topic2  -5.9024   1.0765
51             brasil   6223.531486  43613.595827   Topic2  -5.4860  -0.5243
526               sao   4912.691973  21317.887707   Topic2  -5.7225  -0.0449
773           militar   3548.051662   9894.018697   Topic2  -6.0479   0.3972

[152 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
46         1  0.247877        1
46         2  0.752106        1
18145      1  0.000707      101
18145      2  0.999803      101
4508       1  0.000637      111
...      ...       ...      ...
692        2  0.000131    votos
39         1  0.000409    youtu
39         2  0.999605    youtu
17         1  0.065398  youtube
17         2  0.934614  youtube

[214 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [21]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 1.0
IRBO médio: 1.0

Perplexidade:  -8.479327591480994


In [22]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppSemFiltro_c_v.csv', index=False, encoding='utf-8')


In [23]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.239279  0.026237       1        1  10.379531
15    -0.231022 -0.082353       2        1  10.141031
19    -0.057431  0.136445       3        1   9.876264
14    -0.222893 -0.099059       4        1   8.882574
2     -0.114084  0.163450       5        1   5.939226
7     -0.189769 -0.074851       6        1   5.770731
10    -0.173671  0.010846       7        1   5.383563
20     0.129098  0.186066       8        1   4.297261
3     -0.117593  0.149081       9        1   4.216571
1     -0.088728  0.141364      10        1   4.216139
12     0.108439  0.074814      11        1   3.385494
11    -0.032651 -0.067579      12        1   3.205582
21     0.187576  0.037954      13        1   2.604467
13     0.137773  0.089086      14        1   2.333947
16     0.096027 -0.071489      15        1   2.206154
17     0.057115 -0.031982      16        1   2.198876
5      0.194812  0.053635      17        1   2.145582
18     0.044248 -0.014396      18        1   2.140609
9      0.017890 -0.141905      19        1   2.133769
22     0.031834 -0.156775      20        1   2.062923
23     0.003913 -0.153010      21        1   1.974787
0      0.106033  0.102551      22        1   1.918628
6      0.116994 -0.145928      23        1   1.553053
4      0.235370 -0.132202      24        1   1.033237, topic_info=            Term          Freq         Total Category  logprob  loglift
3607          13  61934.000000  61934.000000  Default  30.0000  30.0000
4005          55  47739.000000  47739.000000  Default  29.0000  29.0000
39         youtu  43192.000000  43192.000000  Default  28.0000  28.0000
8480       phone  44414.000000  44414.000000  Default  27.0000  27.0000
9           lula  53348.000000  53348.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
13444  impedindo    452.586563    598.466054  Topic24  -4.9574   4.2931
2754        dedo    832.486771   1587.320827  Topic24  -4.3480   3.9271
4622       monte    537.385793    790.404566  Topic24  -4.7857   4.1866
2047      homens    742.777395   1689.712727  Topic24  -4.4620   3.7506
6119     gabriel    397.944344    954.651382  Topic24  -5.0861   3.6974

[1661 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
27243     24  0.998592  0000
7082       2  0.107986    04
7082       5  0.036795    04
7082       7  0.000800    04
7082      11  0.836691    04
...      ...       ...   ...
5113       3  0.189103  zona
5113       8  0.130833  zona
5113      11  0.122404  zona
5113      16  0.001466  zona
5113      17  0.397262  zona

[5615 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 16, 20, 15, 3, 8, 11, 21, 4, 2, 13, 12, 22, 14, 17, 18, 6, 19, 10, 23, 24, 1, 7, 5])

In [24]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 0.775
IRBO médio: 0.9642

Perplexidade:  -10.520764588643944


In [25]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppSemFiltro_c_uci.csv', index=False, encoding='utf-8')

In [26]:

pyLDAvis.enable_notebook() 

# Escolher a métrica para visualizar
chosen_metric = 'c_npmi'  # ou 'c_v', 'c_uci', 'u_mass'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis  

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.244426  0.056678       1        1  9.679230
8     -0.253016  0.123176       2        1  9.520831
1     -0.043562 -0.043092       3        1  8.742401
15    -0.233879  0.061956       4        1  8.616216
3     -0.170646 -0.112422       5        1  7.760325
5     -0.183826 -0.003341       6        1  5.578357
11    -0.118118 -0.083322       7        1  5.402115
16    -0.056602 -0.190777       8        1  4.374421
0      0.104899 -0.030098       9        1  4.260242
20     0.025645  0.082082      10        1  4.001744
10    -0.061587  0.123432      11        1  3.918366
4      0.062079 -0.201984      12        1  3.519027
2     -0.082574  0.020940      13        1  2.756262
7      0.190673  0.017541      14        1  2.572294
18     0.069379 -0.260491      15        1  2.412496
22     0.087503  0.091048      16        1  2.291683
19     0.095446  0.002329      17        1  2.261646
14     0.081796  0.081427      18        1  2.179884
6      0.139138  0.098367      19        1  2.022525
23     0.168348 -0.042140      20        1  1.949658
21     0.096596  0.170213      21        1  1.766326
9      0.032821 -0.065948      22        1  1.687407
12     0.102985  0.009262      23        1  1.490152
17     0.190929  0.095167      24        1  1.236394, topic_info=          Term          Freq         Total Category  logprob  loglift
3607        13  55219.000000  55219.000000  Default  30.0000  30.0000
4005        55  47322.000000  47322.000000  Default  29.0000  29.0000
8480     phone  46464.000000  46464.000000  Default  28.0000  28.0000
39       youtu  40340.000000  40340.000000  Default  27.0000  27.0000
9         lula  59289.000000  59289.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
443    vigilia    442.671642    940.971498  Topic24  -5.1591   3.6389
3562  divulgue    432.819802    889.153190  Topic24  -5.1816   3.6730
445    vontade    507.017375   2204.936446  Topic24  -5.0234   2.9231
413       cair    423.719260   1214.064669  Topic24  -5.2028   3.3403
566     senado    448.012182   2745.125628  Topic24  -5.1471   2.5802

[1675 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
8227       3  0.125068     0
8227       7  0.874190     0
27243     20  0.997141  0000
7082       5  0.001864    04
7082       7  0.147850    04
...      ...       ...   ...
5113       1  0.021513  zona
5113       3  0.782301  zona
5113       4  0.000391  zona
5113       7  0.155678  zona
5113       9  0.039506  zona

[5779 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 9, 2, 16, 4, 6, 12, 17, 1, 21, 11, 5, 3, 8, 19, 23, 20, 15, 7, 24, 22, 10, 13, 18])

In [27]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus))

Diversidade de Tópicos: 0.7667
IRBO médio: 0.9717

Perplexidade:  -10.547148837972854


In [28]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppSemFiltro_c_npmi.csv', index=False, encoding='utf-8')


In [29]:
df_geral.shape

(254480, 21)

# Modelando os Tópicos com filtro religioso

In [30]:
palavras_religiosas = [
    "deus", "jesus", "misericordia", "davi",
    "salomao", "reino", "templo", "conservador",
    "pentecostal", "rcc", "renovacao", "carismatic",
    "paulo ricardo", "bernardo kuster", "herege", "ateu",
    "jerico", "heresia"
]


In [31]:
# Função para verificar se uma palavra está relacionada à religião
def relacionada_religiao(word):
    word_lower = word.lower()
    palavras_religiosas_lower = [palavra.lower() for palavra in palavras_religiosas]

    # Verificando se alguma palavra da lista de palavras religiosas está presente
    return any(palavra in word_lower for palavra in palavras_religiosas_lower)


df_religiao = df_geral[df_geral['text_processed'].apply(lambda x: relacionada_religiao(x))]



In [32]:
len(df_religiao)

15507

In [33]:
def processar_texto(texto):
    texto = texto.lower()
    palavras_texto = token_espaco.tokenize(texto)
    palavras_texto = token_pontuacao.tokenize(' '.join(palavras_texto))
    return [palavra for palavra in palavras_texto ]

texto_processado_religiao = df_religiao['text_processed'].apply(processar_texto)

In [34]:
# Criar dicionário
id2word_religiao = corpora.Dictionary(texto_processado_religiao)
# Criar o Corpus: A Frequência das palavras nos documentos
corpus_religiao = [id2word_religiao.doc2bow(text) for text in texto_processado_religiao]

In [35]:
# tamanho do corpus
len(corpus_religiao)

15507

In [36]:

# Função para determinar a melhor quantidade de tópicos para a modelagem
def compute_coherence_values(dictionary, corpus_religiao, texts, coherence, limit=30, start=2, step=5):
    coherence_values = []
    model_list = []
    topic_range = list(range(start, limit, step))

    for num_topics in tqdm(topic_range, desc=f"Calculando coerência ({coherence})"):
        model = LdaMulticore(corpus=corpus_religiao, id2word=dictionary, num_topics=num_topics, passes=5, workers=4)
        model_list.append(model)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    optimal_index = coherence_values.index(max(coherence_values))
    optimal_num_topics = topic_range[optimal_index]
    return optimal_num_topics, model_list, coherence_values

# Calcular os valores de coerência para diferentes métricas
def calculate_coherence_for_metrics(dictionary, corpus_religiao, texts, metrics, start=2, limit=30, step=2):
    coherence_results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_values(
            dictionary=dictionary,
            corpus_religiao=corpus_religiao,
            texts=texts,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        coherence_results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return coherence_results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

# Calcular os valores de coerência 
coherence_results = calculate_coherence_for_metrics(
    dictionary=id2word_religiao,
    corpus_religiao=corpus_religiao,
    texts=texto_processado_religiao,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)

# Plotar os resultados de coerência para cada métrica
for metric in metrics:
    x = list(range(2, 30, 2))
    coherence_values = coherence_results[metric]['coherence_values']
    plt.plot(x, coherence_values, marker='o', label=metric.upper())

plt.xlabel("Número de Tópicos")
plt.ylabel("Score de Coerência")
plt.legend(loc='best')
plt.title("Coerência por Métrica")
plt.grid(True)
plt.tight_layout()
plt.show()

# Mostrar os resultados de coerência para cada métrica
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {coherence_results[metric]['num_topics']}")


Calculando coerência (c_npmi): 100%|██████████| 14/14 [07:46<00:00, 33.29s/it]



Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.3786
Qtd. tópicos = 4 -> Coerência = 0.3981
Qtd. tópicos = 6 -> Coerência = 0.4502
Qtd. tópicos = 8 -> Coerência = 0.4643
Qtd. tópicos = 10 -> Coerência = 0.5019
Qtd. tópicos = 12 -> Coerência = 0.4944
Qtd. tópicos = 14 -> Coerência = 0.4761
Qtd. tópicos = 16 -> Coerência = 0.4939
Qtd. tópicos = 18 -> Coerência = 0.4822
Qtd. tópicos = 20 -> Coerência = 0.5013
Qtd. tópicos = 22 -> Coerência = 0.5246
Qtd. tópicos = 24 -> Coerência = 0.51
Qtd. tópicos = 26 -> Coerência = 0.4989
Qtd. tópicos = 28 -> Coerência = 0.538
Melhor número de tópicos segundo C_V: 28

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -2.5548
Qtd. tópicos = 4 -> Coerência = -2.1729
Qtd. tópicos = 6 -> Coerência = -1.8445
Qtd. tópicos = 8 -> Coerência = -2.1146
Qtd. tópicos = 10 -> Coerência = -1.8808
Qtd. tópicos = 12 -> Coerência = -1.8
Qtd. tópicos = 14 -> Coerência = -1.6317
Qtd. tópicos = 16 -> Coerência = -1.6752
Qtd. tópicos

In [37]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'u_mass'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.051383  0.154447       1        1  15.198153
1      0.087612 -0.122515       2        1  14.476038
0      0.038267  0.238526       3        1   7.846878
9      0.091289 -0.041287       4        1   7.180543
3      0.054787 -0.043316       5        1   7.171261
12     0.086619 -0.000265       6        1   7.133364
13     0.096520 -0.046577       7        1   7.019117
11     0.092150 -0.006170       8        1   6.556037
2     -0.459720 -0.018612       9        1   5.245898
10     0.061011  0.073329      10        1   5.234095
7      0.053435 -0.097599      11        1   4.925071
6      0.066206 -0.034627      12        1   4.610461
8     -0.383435 -0.006510      13        1   4.163175
4      0.063877 -0.048824      14        1   3.239910, topic_info=        Term          Freq         Total Category  logprob  loglift
1298      55  29026.000000  29026.000000  Default  30.0000  30.0000
4633   phone  28228.000000  28228.000000  Default  29.0000  29.0000
812       20   8487.000000   8487.000000  Default  28.0000  28.0000
45       nao  28446.000000  28446.000000  Default  27.0000  27.0000
375     lula  10226.000000  10226.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
90     video    209.473934   1370.066040  Topic14  -5.6375   1.5516
307   melhor    208.555157   1333.333041  Topic14  -5.6419   1.5744
290    todos    231.902778   8550.016040  Topic14  -5.5358  -0.1777
807    fazer    216.442116   2911.904031  Topic14  -5.6048   0.8304
124        2    207.284988   1936.575627  Topic14  -5.6480   1.1950

[1205 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
3076       1  0.051246     01
3076       2  0.017082     01
3076       3  0.008541     01
3076       4  0.025623     01
3076       5  0.059787     01
...      ...       ...    ...
1489      10  0.007961   zica
1489      11  0.007961   zica
1489      12  0.007961   zica
1489      14  0.007961   zica
21170      1  0.928518  zumba

[8181 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 1, 10, 4, 13, 14, 12, 3, 11, 8, 7, 9, 5])

In [38]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.55
IRBO médio: 0.8101

Perplexidade:  -7.677445759060923


In [39]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroReligioso_u_mass.csv', index=False, encoding='utf-8')


In [40]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_v'  # ou 'u_mass', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
22    -0.469160  0.169334       1        1  9.173539
16    -0.006265 -0.175220       2        1  8.589585
24     0.110155  0.099488       3        1  6.580321
12     0.075460  0.056490       4        1  5.688146
4     -0.026221 -0.195079       5        1  5.679879
17     0.093830  0.063014       6        1  5.297270
15    -0.042862 -0.010449       7        1  5.225371
27     0.092441  0.048076       8        1  4.461882
18     0.067925  0.055438       9        1  4.347747
25     0.006429 -0.074230      10        1  4.246041
26     0.076144  0.041302      11        1  3.892440
2      0.076516  0.041209      12        1  3.477800
3      0.046804  0.139862      13        1  3.266936
7     -0.003992 -0.065530      14        1  3.157440
13    -0.066010 -0.026420      15        1  2.921944
11     0.063588 -0.020533      16        1  2.664364
8      0.025508  0.021352      17        1  2.599332
6      0.059659  0.054915      18        1  2.227609
1     -0.026235  0.098716      19        1  2.211810
20    -0.139943 -0.172478      20        1  2.130222
14     0.042844  0.076254      21        1  1.796572
23    -0.031024 -0.161247      22        1  1.620091
9     -0.008983  0.008417      23        1  1.606419
19    -0.134915  0.043646      24        1  1.581489
10    -0.011698 -0.124261      25        1  1.568862
21     0.045669  0.043520      26        1  1.540274
0      0.027609 -0.043844      27        1  1.444783
5      0.056727  0.008258      28        1  1.001834, topic_info=           Term          Freq         Total Category  logprob  loglift
1298         55  30287.000000  30287.000000  Default  30.0000  30.0000
4633      phone  29391.000000  29391.000000  Default  29.0000  29.0000
375        lula   9623.000000   9623.000000  Default  28.0000  28.0000
317   bolsonaro  11361.000000  11361.000000  Default  27.0000  27.0000
812          20   8814.000000   8814.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
26         cada     81.961203   1786.545453  Topic28  -5.4022   1.5215
932    esquerda     78.506111   1964.468932  Topic28  -5.4452   1.3835
317   bolsonaro     80.175393  11361.881218  Topic28  -5.4242  -0.3505
69      vontade     71.125258    803.957165  Topic28  -5.5440   2.1782
61         sera     72.878277   3097.769833  Topic28  -5.5196   0.8537

[2308 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
48527      1  0.972154      0523
48527      9  0.002606      0523
48527     19  0.013032      0523
48527     24  0.002606      0523
11389     18  0.916882  06092018
...      ...       ...       ...
1489      22  0.007386      zica
1489      28  0.007386      zica
48409     25  0.840623       zio
21170     19  0.069443     zumba
21170     21  0.833314     zumba

[18071 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[23, 17, 25, 13, 5, 18, 16, 28, 19, 26, 27, 3, 4, 8, 14, 12, 9, 7, 2, 21, 15, 24, 10, 20, 11, 22, 1, 6])

In [41]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.4786
IRBO médio: 0.8162

Perplexidade:  -7.587056074493184


In [42]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroReligioso_c_v.csv', index=False, encoding='utf-8')


In [43]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_uci'  # ou 'c_v', 'u_mass', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3     -0.076908 -0.099382       1        1  8.843842
18    -0.086828 -0.058442       2        1  8.328509
20    -0.069990 -0.091801       3        1  7.287657
12     0.011151  0.180767       4        1  6.372165
8     -0.048770  0.153385       5        1  5.272635
15     0.486191 -0.010855       6        1  5.216811
13    -0.028884  0.112807       7        1  5.042759
19    -0.009888  0.168246       8        1  4.926327
16    -0.057686 -0.070634       9        1  4.807507
21    -0.045138  0.148516      10        1  4.561688
5     -0.040819 -0.091250      11        1  4.553074
7      0.362930 -0.056928      12        1  4.257762
10    -0.024302 -0.165393      13        1  4.056489
0     -0.072997 -0.058146      14        1  3.753105
2     -0.003000  0.155904      15        1  3.624512
4     -0.043934 -0.082867      16        1  3.619586
6     -0.053171 -0.021713      17        1  3.220911
9     -0.063891 -0.054755      18        1  3.119135
11    -0.000343 -0.060339      19        1  2.934672
17    -0.070571  0.007495      20        1  2.814651
14    -0.025468  0.027108      21        1  1.976830
1     -0.037684 -0.031725      22        1  1.409374, topic_info=       Term          Freq         Total Category  logprob  loglift
1298     55  28508.000000  28508.000000  Default  30.0000  30.0000
4633  phone  27867.000000  27867.000000  Default  29.0000  29.0000
812      20   8555.000000   8555.000000  Default  28.0000  28.0000
375    lula  10074.000000  10074.000000  Default  27.0000  27.0000
475   youtu   4126.000000   4126.000000  Default  26.0000  26.0000
...     ...           ...           ...      ...      ...      ...
945    tudo    280.212394   5521.612649  Topic22  -4.5142   1.2811
516   vamos    244.274437   4396.514911  Topic22  -4.6514   1.3717
45      nao    266.916654  28535.735095  Topic22  -4.5628  -0.4100
3637  maior    162.391436   2280.211923  Topic22  -5.0597   1.6200
23    assim    152.876535   2911.854298  Topic22  -5.1201   1.3151

[1853 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
11389      5  0.923595  06092018
11389     20  0.024305  06092018
122        1  0.019496         1
122        2  0.023549         1
122        3  0.048257         1
...      ...       ...       ...
1489       2  0.007869      zica
1489       4  0.936401      zica
1489       7  0.007869      zica
1489      11  0.007869      zica
1489      12  0.015738      zica

[13267 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 19, 21, 13, 9, 16, 14, 20, 17, 22, 6, 8, 11, 1, 3, 5, 7, 10, 12, 18, 15, 2])

In [44]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.4727
IRBO médio: 0.8153

Perplexidade:  -7.638928866533885


In [45]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroReligioso_c_uci.csv', index=False, encoding='utf-8')


In [46]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_npmi'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_religiao, id2word_religiao)
vis  


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.063600  0.041390       1        1  10.414385
20     0.480611  0.164525       2        1   9.186537
19     0.039120 -0.209609       3        1   9.178198
0     -0.059915  0.065930       4        1   5.973571
18     0.003440 -0.167427       5        1   5.915491
21     0.121403 -0.098828       6        1   4.458839
2     -0.028054 -0.004970       7        1   4.425207
5     -0.073677  0.052007       8        1   4.214377
11    -0.063993 -0.012756       9        1   4.203993
24    -0.060342  0.114369      10        1   3.506144
13    -0.072721  0.150982      11        1   3.402446
6     -0.040692 -0.004450      12        1   3.104365
15    -0.064021  0.012417      13        1   2.786503
4      0.037294 -0.016944      14        1   2.669240
22     0.021274  0.011608      15        1   2.666227
7     -0.030233  0.004031      16        1   2.611886
3      0.074230 -0.156282      17        1   2.603168
25    -0.067037  0.105729      18        1   2.597534
8     -0.044082  0.039382      19        1   2.557616
1     -0.043895  0.036027      20        1   2.469893
16    -0.055830  0.049771      21        1   2.331975
23    -0.030802  0.011260      22        1   2.309307
10    -0.022379 -0.117877      23        1   2.154877
14    -0.029250 -0.012207      24        1   1.843540
17     0.001337  0.050103      25        1   1.480026
12     0.071815 -0.108180      26        1   0.934657, topic_info=       Term          Freq         Total Category  logprob  loglift
1298     55  30069.000000  30069.000000  Default  30.0000  30.0000
4633  phone  29153.000000  29153.000000  Default  29.0000  29.0000
45      nao  28265.000000  28265.000000  Default  28.0000  28.0000
812      20   8717.000000   8717.000000  Default  27.0000  27.0000
475   youtu   3976.000000   3976.000000  Default  26.0000  26.0000
...     ...           ...           ...      ...      ...      ...
724       r     59.286845   3195.556862  Topic26  -5.6566   0.6856
1328  bolsa     48.068320    474.282078  Topic26  -5.8664   2.3836
1100  nunca     53.113231   2082.482596  Topic26  -5.7666   1.0039
33     deus     57.793424  16966.245686  Topic26  -5.6821  -1.0094
463    pais     54.253117   5017.946247  Topic26  -5.7454   0.1456

[2146 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
3079       1  0.008275     04
3079       2  0.775749     04
3079       3  0.014481     04
3079       5  0.002069     04
3079       6  0.020687     04
...      ...       ...    ...
1489      19  0.007084   zica
1489      20  0.007084   zica
1489      22  0.007084   zica
1489      26  0.014168   zica
21170     26  0.906495  zumba

[17206 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 21, 20, 1, 19, 22, 3, 6, 12, 25, 14, 7, 16, 5, 23, 8, 4, 26, 9, 2, 17, 24, 11, 15, 18, 13])

In [47]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_religiao))

Diversidade de Tópicos: 0.4962
IRBO médio: 0.8036

Perplexidade:  -7.584643658672211


In [48]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroReligioso_c_npmi.csv', index=False, encoding='utf-8')


## Modelando os tópicos com filtro de inclusão religioso e exclusão de termas políticos

In [49]:
palavras_politicas = [ "lula", "bolsonaro", "pt", "pl", "stf", "patria", "55", "22", "13", "senadores", "lulaladrao",
                       "urnas", "alexandre", "moraes", "comunismo", "eleicao", "eleicoes", "esquerda", "direita",
                         "presidente", "tse", "fraude", "voto", "turno", "ministro"]

In [50]:
def retirar_mensagens_com_palavras_politicas(word):
    word_lower = word.lower()
    palavras_politica_lower = [palavra.lower() for palavra in palavras_politicas]

    return any(palavra in word_lower for palavra in palavras_politica_lower)


df_politico = df_religiao[~df_religiao['text_processed'].apply(lambda x: retirar_mensagens_com_palavras_politicas(x))]



In [51]:
df_politico.shape

(5167, 21)

In [52]:
df_politico.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5,text_processed
165,2022-10-10 18:18:11,f9fd00ad20b8d4a40ad9fd2d7c64cb40,e110071613239754d38878f7e046e95b,NaN,NaN,NaN,False,False,False,Deus perdoa porque eles Não sabem que fazem e ...,...,2022-10-10 18:18:11.689928,0.4404,NaN,44C0D9A25FA1F439172A6F67EC88B3ED,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,deus perdoa porque nao sabem fazem falam nao t...
215,2022-10-10 16:57:12,3d4e77cea17b1812993b55543e81400e,d01bfa0f0d31fa1307e7aeb9d3c3870e,NaN,NaN,https://www.portalcidade.news/imprensa-tendenc...,False,True,False,Imprensa tendenciosa age novamente e ataca pre...,...,2022-10-10 16:57:12.981007,0.0000,NaN,0993FD7ABC0CA5A21AED8DA493993B73,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,imprensa tendenciosa age novamente ataca prefe...
218,2022-10-10 18:34:17,71d2b615339a7c53dcf2427491da5d36,e110071613239754d38878f7e046e95b,NaN,NaN,NaN,False,False,False,Abre o esgoto que vc chama de boca porra e adm...,...,2022-10-10 18:34:17.860259,-0.7351,NaN,712BCD08CC4F7EA5514AC4A7BD6C8B9B,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,abre esgoto vc chama boca porra admita deus ma...
239,2022-10-10 18:35:14,f9fd00ad20b8d4a40ad9fd2d7c64cb40,e110071613239754d38878f7e046e95b,NaN,NaN,NaN,False,False,False,Deus é vida. Morreu na cruz pra dar vida. Ou s...,...,2022-10-10 18:35:14.825606,-0.3612,0.267817,9D8849865FB72F726E9B182937590C9C,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,deus vida morreu cruz pra dar vida vc nao acre...
333,2022-10-10 18:37:01,0842f1acc471ab358a596e0c34006c21,e110071613239754d38878f7e046e95b,NaN,NaN,NaN,False,False,False,A gente pode manda em Deus ??? Ser ele tem um ...,...,2022-10-10 18:37:01.781209,0.5514,0.116997,EABCED2ADAF7A65F48B7CD567F28534D,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,gente pode manda deus decreto tal cidade pode ...


In [53]:
def processar_texto(texto):
    texto = texto.lower()
    palavras_texto = token_espaco.tokenize(texto)
    palavras_texto = token_pontuacao.tokenize(' '.join(palavras_texto))
    return [palavra for palavra in palavras_texto ]

texto_processado_politico = df_politico['text_processed'].apply(processar_texto)

In [54]:
# Criar dicionário
id2word_politico = corpora.Dictionary(texto_processado_politico)
# Criar o Corpus: A Frequência das palavras nos documentos
corpus_politico = [id2word_politico.doc2bow(text) for text in texto_processado_politico]

In [55]:

# Função para determinar a melhor quantidade de tópicos para a modelagem
def compute_coherence_values(dictionary, corpus_politico, texts, coherence, limit=30, start=2, step=5):
    coherence_values = []
    model_list = []
    topic_range = list(range(start, limit, step))

    for num_topics in tqdm(topic_range, desc=f"Calculando coerência ({coherence})"):
        model = LdaMulticore(corpus=corpus_politico, id2word=dictionary, num_topics=num_topics, passes=5, workers=4)
        model_list.append(model)

        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    optimal_index = coherence_values.index(max(coherence_values))
    optimal_num_topics = topic_range[optimal_index]
    return optimal_num_topics, model_list, coherence_values

# Calcular os valores de coerência para diferentes métricas
def calculate_coherence_for_metrics(dictionary, corpus_politico, texts, metrics, start=2, limit=30, step=2):
    coherence_results = {}
    for metric in metrics:
        num_topics, model_list, coherence_values = compute_coherence_values(
            dictionary=dictionary,
            corpus_politico=corpus_politico,
            texts=texts,
            coherence=metric,
            start=start,
            limit=limit,
            step=step
        )
        coherence_results[metric] = {
            'num_topics': num_topics,
            'model_list': model_list,
            'coherence_values': coherence_values
        }
    return coherence_results

metrics = ['c_v', 'u_mass', 'c_uci', 'c_npmi']

# Calcular os valores de coerência 
coherence_results = calculate_coherence_for_metrics(
    dictionary=id2word_politico,
    corpus_politico=corpus_politico,
    texts=texto_processado_politico,
    metrics=metrics,
    start=2,
    limit=30,
    step=2
)

# Plotar os resultados de coerência para cada métrica
for metric in metrics:
    x = list(range(2, 30, 2))
    coherence_values = coherence_results[metric]['coherence_values']
    plt.plot(x, coherence_values, marker='o', label=metric.upper())

plt.xlabel("Número de Tópicos")
plt.ylabel("Score de Coerência")
plt.legend(loc='best')
plt.title("Coerência por Métrica")
plt.grid(True)
plt.tight_layout()
plt.show()

# Mostrar os resultados de coerência para cada métrica
for metric in metrics:
    print(f"\nResultados de Coerência para {metric.upper()}:")
    for m, cv in zip(range(2, 30, 2), coherence_results[metric]['coherence_values']):
        print(f"Qtd. tópicos = {m} -> Coerência = {round(cv, 4)}")
    print(f"Melhor número de tópicos segundo {metric.upper()}: {coherence_results[metric]['num_topics']}")


Calculando coerência (c_npmi): 100%|██████████| 14/14 [03:38<00:00, 15.60s/it]



Resultados de Coerência para C_V:
Qtd. tópicos = 2 -> Coerência = 0.4809
Qtd. tópicos = 4 -> Coerência = 0.4688
Qtd. tópicos = 6 -> Coerência = 0.4126
Qtd. tópicos = 8 -> Coerência = 0.4446
Qtd. tópicos = 10 -> Coerência = 0.4367
Qtd. tópicos = 12 -> Coerência = 0.4323
Qtd. tópicos = 14 -> Coerência = 0.4326
Qtd. tópicos = 16 -> Coerência = 0.4526
Qtd. tópicos = 18 -> Coerência = 0.4123
Qtd. tópicos = 20 -> Coerência = 0.43
Qtd. tópicos = 22 -> Coerência = 0.4476
Qtd. tópicos = 24 -> Coerência = 0.4364
Qtd. tópicos = 26 -> Coerência = 0.4223
Qtd. tópicos = 28 -> Coerência = 0.4302
Melhor número de tópicos segundo C_V: 2

Resultados de Coerência para U_MASS:
Qtd. tópicos = 2 -> Coerência = -2.9741
Qtd. tópicos = 4 -> Coerência = -3.5174
Qtd. tópicos = 6 -> Coerência = -2.9018
Qtd. tópicos = 8 -> Coerência = -3.4481
Qtd. tópicos = 10 -> Coerência = -3.3102
Qtd. tópicos = 12 -> Coerência = -3.2486
Qtd. tópicos = 14 -> Coerência = -3.4059
Qtd. tópicos = 16 -> Coerência = -3.7661
Qtd. tópi

In [56]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'u_mass'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.027051 -0.000193       1        1  29.732481
0     -0.028015 -0.040589       2        1  18.426332
2     -0.024418 -0.013179       3        1  16.086527
3     -0.018650 -0.061482       4        1  13.592586
5     -0.048357  0.100193       5        1  12.830061
1      0.146492  0.015251       6        1   9.332013, topic_info=        Term         Freq        Total Category  logprob  loglift
1       deus  4651.000000  4651.000000  Default  30.0000  30.0000
4        nao  2445.000000  2445.000000  Default  29.0000  29.0000
112    todos   828.000000   828.000000  Default  28.0000  28.0000
881    acima   278.000000   278.000000  Default  27.0000  27.0000
13205  nunes   101.000000   101.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
112    todos    37.828280   828.151535   Topic6  -5.6613  -0.7144
142     amor    32.015404   371.279599   Topic6  -5.8281  -0.0790
237    vamos    35.477305   668.808809   Topic6  -5.7254  -0.5649
148      ter    27.992352   220.410806   Topic6  -5.9624   0.3082
410     hoje    27.197384   192.000538   Topic6  -5.9912   0.4173

[508 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
909       6  0.870145   0500
241       1  0.065096    100
241       2  0.878793    100
241       3  0.032548    100
716       6  0.854648  10700
...     ...       ...    ...
254       2  0.261740  youtu
254       3  0.025745  youtu
254       4  0.111561  youtu
254       5  0.090107  youtu
254       6  0.128724  youtu

[1221 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 3, 4, 6, 2])

In [57]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.3833
IRBO médio: 0.3597

Perplexidade:  -8.034230993157026


In [58]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroPolitico_u_mass.csv', index=False, encoding='utf-8')


In [59]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_v'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.051274  0.0       1        1  62.681959
0     -0.051274  0.0       2        1  37.318041, topic_info=        Term         Freq        Total Category  logprob  loglift
4        nao  2409.000000  2409.000000  Default  30.0000  30.0000
841     amem   374.000000   374.000000  Default  29.0000  29.0000
886    estao   247.000000   247.000000  Default  28.0000  28.0000
881    acima   274.000000   274.000000  Default  27.0000  27.0000
1697   ifood    88.000000    88.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
766       fe   104.099952   292.802002   Topic2  -6.0350  -0.0485
54      pode    98.247687   231.306838   Topic2  -6.0929   0.1294
107   sempre    99.324885   272.030261   Topic2  -6.0820  -0.0218
189   cristo   102.090588   426.715219   Topic2  -6.0545  -0.4446
266     povo   100.121214   473.360305   Topic2  -6.0740  -0.5678

[200 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3644      1  0.101748       15
3644      2  0.915735       15
5311      1  0.975416      1os
5311      2  0.054190      1os
1685      1  0.112014       50
...     ...       ...      ...
525       2  0.162921  vitoria
339       1  0.706350     voce
339       2  0.293747     voce
1507      1  0.904216  vontade
1507      2  0.092740  vontade

[303 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [60]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.65
IRBO médio: 0.1865

Perplexidade:  -7.960195138926245


In [61]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroPolitico_c_v.csv', index=False, encoding='utf-8')

In [62]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_uci'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.050467  0.0       1        1  51.589119
1     -0.050467  0.0       2        1  48.410881, topic_info=       Term         Freq        Total Category  logprob  loglift
259   jesus  1264.000000  1264.000000  Default  30.0000  30.0000
108  senhor  1203.000000  1203.000000  Default  29.0000  29.0000
79      dia   785.000000   785.000000  Default  28.0000  28.0000
4       nao  2411.000000  2411.000000  Default  27.0000  27.0000
73      bom   471.000000   471.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
259   jesus   242.385703  1264.106764   Topic2  -5.4501  -0.9261
231   sobre   128.161023   242.118796   Topic2  -6.0873   0.0893
137     vai   165.762271   552.135855   Topic2  -5.8300  -0.4778
886   estao   130.391572   255.579269   Topic2  -6.0701   0.0525
79      dia   177.801040   785.400109   Topic2  -5.7599  -0.7601

[195 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
241        1  0.097852     100
241        2  0.913282     100
3644       1  0.899162      15
3644       2  0.094649      15
12778      1  0.925894      64
...      ...       ...     ...
339        2  0.472811    voce
1490       1  0.892655  voltar
1490       2  0.109865  voltar
254        1  0.412359   youtu
254        2  0.588470   youtu

[298 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [63]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.75
IRBO médio: 0.3815

Perplexidade:  -7.956362486902038


In [64]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroPolitico_c_uci.csv', index=False, encoding='utf-8')


In [65]:

pyLDAvis.enable_notebook()  

# Escolher a métrica para visualizar
chosen_metric = 'c_npmi'  # ou 'c_v', 'c_uci', 'c_npmi'

# Obter o melhor modelo já treinado
best_index = coherence_results[chosen_metric]['coherence_values'].index(
    max(coherence_results[chosen_metric]['coherence_values'])
)
lda_model = coherence_results[chosen_metric]['model_list'][best_index]

# Gerar a visualização
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus_politico, id2word_politico)
vis  



PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.052504  0.0       1        1  63.728022
0     -0.052504  0.0       2        1  36.271978, topic_info=       Term         Freq        Total Category  logprob  loglift
259   jesus  1208.000000  1208.000000  Default  30.0000  30.0000
108  senhor  1169.000000  1169.000000  Default  29.0000  29.0000
265    nome   518.000000   518.000000  Default  28.0000  28.0000
841    amem   345.000000   345.000000  Default  27.0000  27.0000
189  cristo   405.000000   405.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
47     vida   149.002635   472.684077   Topic2  -5.6480  -0.1403
339    voce   156.672734   590.766620   Topic2  -5.5978  -0.3131
254   youtu   107.991765   230.529276   Topic2  -5.9699   0.2558
6    porque   117.485788   394.924863   Topic2  -5.8856  -0.1983
266    povo   115.424637   471.482687   Topic2  -5.9033  -0.3931

[193 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
241        1  0.957899     100
241        2  0.061800     100
12778      1  0.066953      64
12778      2  0.937347      64
13200      1  0.933648     673
...      ...       ...     ...
1490       2  0.824075  voltar
858        1  0.966909   votar
858        2  0.037918   votar
254        1  0.533555   youtu
254        2  0.468487   youtu

[293 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [66]:
div = topic_diversity(lda_model)
print(f"Diversidade de Tópicos: {round(div, 4)}")

irbo = compute_irbo(lda_model)
print(f"IRBO médio: {round(irbo, 4)}")

# Computar a perplexidade do modelo (quanto menor, melhor)
print('\nPerplexidade: ', lda_model.log_perplexity(corpus_politico))

Diversidade de Tópicos: 0.65
IRBO médio: 0.3555

Perplexidade:  -7.955651257789668


In [67]:
# Extrair tópicos (palavras e pesos)
topics_data = []
for topic_id, topic in lda_model.show_topics(formatted=False):
    for word, weight in topic:
        topics_data.append({
            'topic_id': topic_id,
            'word': word,
            'weight': weight
        })

# Criar DataFrame
topics_df = pd.DataFrame(topics_data)


# Salvar em CSV
topics_df.to_csv('./resultados_lda/wpp/lda_model_wppComFiltroPolitico_c_npmi.csv', index=False, encoding='utf-8')
